In [ ]:
import os
import pickle
import numpy as np
import json
from myEstimator import gd_estimator, mymcmc_estimator2, ML_estimator
from scipy.stats import beta

folder = '/Users/anjiefang/Desktop/CIKM2017_select_best/'
INTERVALS = [30,60,120]
trainning_size = [0.5, 0.6, 0.8]
estimator_name = ['GDE', 'MLE', 'MCE']

fold_num = 5
partition_num = 1000
est_method = 'BFGS'

errors = [[[[] for e in estimator_name] for t in trainning_size ] for i in INTERVALS]

def rmse(data, dist, para):
    y_true, bins = np.histogram(data, bins=20, density=True)
    bins[0] = 0.0
    bins[-1] = 1.0
    est_a = np.array([(dist.cdf(bins[i + 1], para[0], para[1]) - dist.cdf(bins[i], para[0], para[1])) for i in range(len(bins) - 1)])
    step = np.array([(bins[i + 1] - bins[i]) for i in range(len(bins) - 1)])
    if np.sum(est_a) < 1e-5:
        return 2.0
    tru_a = y_true * step
    return np.sqrt(np.mean((est_a - tru_a)**2))

for i in range(len(INTERVALS)):
    print 'Time Interval: ' + str(INTERVALS[i])
    periods = np.array(pickle.load(open(folder + 'time_period_' + str(INTERVALS[i]) + '/periods.p')))
    for p in range(len(periods)):
        for t in range(len(trainning_size)):
            data_full = periods[p]
            data = np.array([tmp for tmp in data_full if tmp <= trainning_size[t]])
            
            gde = gd_estimator(data = data)
            gd_res = gde.estimate(fold_num=fold_num, partition_num=partition_num)
            err0 = rmse(data_full, beta, gd_res)
            
            mle = ML_estimator(data = data)
            ml_res = mle.estimate(fold_num = fold_num, partition_num = partition_num)
            err1 = rmse(data_full, beta, ml_res)
            
            mce = mymcmc_estimator2(data = data)
            mc_res = mce.estimate(fold_num * 5)
            err2 = rmse(data_full, beta, mc_res)
            
            errors[i][t] = [err0, err1, err2]
        print errors[i]